# Adding More Functions to the Agent (FAQ Usecase)

If you want to learn more about how this library was implemented, here's a workshop where I do this live: https://github.com/alexeygrigorev/rag-agents-workshop

## Data Setup

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


Create the search index:

In [2]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

Define our search function:

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
    )

    return results


Set up the search tool definition:

In [4]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}


## Library Setup

Import the required components:

In [5]:
from toyaikit.llm import OpenAIClient
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner, DisplayingRunnerCallback
from toyaikit.tools import Tools

Initialize the tools:

In [6]:
agent_tools = Tools()
agent_tools.add_tool(search, search_tool)

Setup the instructions for our agent:

In [7]:
instructions = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call
""".strip()


Create a runner configuration:

In [8]:
chat_interface = IPythonChatInterface()

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)


# Running the Agent

In [10]:
callback = DisplayingRunnerCallback(chat_interface)

question = 'how do I install kafka'
loop_result = runner.loop(prompt = question, callback=callback)

In [ ]:
# Token usage and cost, we spent 0.0004 on this request
loop_result.cost

CostInfo(input_cost=0.00018734999999999997, output_cost=0.00027239999999999995, total_cost=0.0004597499999999999)

In [12]:
question = 'but how do I run it in python?'
loop_result = runner.loop(prompt=question, previous_messages=loop_result.new_messages, callback=callback)

In [13]:
loop_result.cost

CostInfo(input_cost=0.00023204999999999998, output_cost=0.00029279999999999996, total_cost=0.0005248499999999999)

### Interactive Chat Mode

In [9]:
runner.run();

You: I just discovered the course. Can I still join it?


You: stop


Chat ended.


## Adding More Functions

In [10]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)


add_entry_tool = {
    "type": "function",
    "name": "add_entry",
    "description": "Add an entry to the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question to be added to the FAQ database",
            },
            "answer": {
                "type": "string",
                "description": "The answer to the question",
            }
        },
        "required": ["question", "answer"],
        "additionalProperties": False
    }
}

Add the new tool to our agent:

In [11]:
agent_tools.add_tool(add_entry, add_entry_tool)

In [12]:
runner.run();

You: how do I do well in module 1?


You: add this back to FAQ


You: stop


Chat ended.


## Putting Tools in One Class

Instead of writing the function definitions, we can add docstrings and typehints (with the help of ChatGPT). ToyAIKit will parse them and use that to create the definitions automatically (and many other tools will too). 

We can also put all the tools in one class, and add all the public methods of this class at once. This also avoids dependency of our tools on global variables.

In [14]:
from typing import Any, Dict, List

class SearchTools:

    def __init__(self, index):
        self.index = index

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the FAQ database for entries matching the given query.
    
        Args:
            query (str): Search query text to look up in the course FAQ.
    
        Returns:
            List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.
        """
        boost = {'question': 3.0, 'section': 0.5}
    
        results = self.index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5,
            output_ids=True
        )
    
        return results

    def add_entry(self, question: str, answer: str) -> None:
        """
        Add a new entry to the FAQ database.
    
        Args:
            question (str): The question to be added to the FAQ database.
            answer (str): The corresponding answer to the question.
        """
        doc = {
            'question': question,
            'text': answer,
            'section': 'user added',
            'course': 'data-engineering-zoomcamp'
        }
        self.index.append(doc)

In [16]:
search_tools = SearchTools(index)

agent_tools = Tools()
agent_tools.add_tools(search_tools)


runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=instructions,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)

runner.run();

You: how do I do well in module 1


You: stop


Chat ended.
